In [17]:
import numpy as np
import pandas as pd 
import random
import TestFunctions as tf

In [14]:
class Solution:
    def __init__(self, dimension, lower_bound, upper_bound):
        self.d = dimension
        self.lower = lower_bound # we will use the same bounds for all parameters
        self.upper = upper_bound
        self.params = np.zeros(self.d) #solution parameters
        self.f = np.inf # objective function evaluation

In [26]:
def generatePopulation(solution : Solution, size):
    population = []
    for _ in range(0, size):
        for i in range(0, len(solution.params)):
            new_solution = Solution(len(solution.params), solution.lower, solution.upper)
            new_solution.params[i] = random.uniform(solution.lower, solution.upper)
        population.append(new_solution)
    return population

def evaluateAll(population, func):
    for solution in population:
        z = func(solution.params)
        solution.f = z
    return population

def DifferentialEvolution(solution, func):
    for i in range(0,len(solution.params)):
        solution.params[i] = random.uniform(solution.lower, solution.upper) # Generate random coordinates for the first time
    #pop = Generate NP random individuals (you can use the class Solution mentioned in Exercise 1)
    population = generatePopulation(solution, 20)
    population = evaluateAll(population, func)
    g = 0
    g_maxim = 200
    F = 0.8
    CR = 0.9
    progressTracker = []
    while g < g_maxim :
        new_popuplation = list(np.copy(population)) # new generation
        for i in range(0, len(population)): # x is also denoted as a target vector
            indices = list(range(len(population)))
            indices.remove(i)
            r1, r2, r3 = np.random.choice(indices, 3, replace=False)
            mutation = (population[r1].params - population[r2].params)*F + population[r3].params # mutation vector. TAKE CARE FOR BOUNDARIES!
            mutation = np.clip(mutation, solution.lower, solution.upper)
            u = np.zeros(len(solution.params)) # trial vector
            j_rnd = np.random.randint(0, len(solution.params))
        for j in range(0, len(solution.params)):
            if np.random.uniform() < CR or j == j_rnd:
                u[j] = mutation[j] # at least 1 parameter should be from a mutation vector v
            else:
                u[j] = population[i].params[j]
        #f_u = Evaluate trial vector u
        f_u = func(u)
        if f_u <= population[i].f: # We always accept a solution with the same fitness as a target vector
            new_x = Solution(len(solution.params), solution.lower, solution.upper)
            new_x.params = u
            new_x.f = f_u
            new_popuplation[i] = new_x
            population = new_popuplation
            progressTracker.append((new_x.params.copy()[0], new_x.params.copy()[1], f_u))
        g += 1
    return progressTracker

In [27]:
funcs = tf.getAllFunctions()
x = Solution(2,funcs[0][1],funcs[0][0])
func = funcs[0][4]
result = DifferentialEvolution(x,func)
#x = result[0]
#func = funcs[0]
#params = result[1]
#plot3DWithAnimation(func[0],func[1],func[2],func[3],func[4],params,'SphereAnnealing.gif')

In [ ]:
result # Clipping is sus

[(0.0, 0.7302925186180627, 0.5333271627495134),
 (0.0, -0.7290610001278468, 0.5315299419074162),
 (0.0, -0.09410728567941273, 0.008856181217946601),
 (0.0, -0.09410728567941273, 0.008856181217946601),
 (0.0, 0.05714294288909283, 0.0032653159220261247),
 (0.0, 0.05714294288909283, 0.0032653159220261247),
 (0.0, -0.04525405856071951, 0.002047929816217031),
 (0.0, -0.04525405856071951, 0.002047929816217031),
 (0.0, -0.04525405856071951, 0.002047929816217031)]

In [3]:
import numpy as np

def differential_evolution(func, bounds, max_iter=1000, pop_size=20, F=0.8, CR=0.9):
    """
    Differential Evolution algorithm for optimization.
    
    Parameters:
    - func: the objective function to minimize
    - bounds: list of tuples, each containing the min and max bounds for each dimension
    - max_iter: maximum number of iterations
    - pop_size: population size
    - F: differential weight (mutation factor)
    - CR: crossover probability
    
    Returns:
    - best_solution: best solution found
    - best_score: objective function value of the best solution
    """
    
    # Initialize population with random values within the given bounds
    dimensions = len(bounds)
    pop = np.random.rand(pop_size, dimensions)
    for i in range(dimensions):
        pop[:, i] = bounds[i][0] + pop[:, i] * (bounds[i][1] - bounds[i][0])
    
    # Evaluate initial population
    fitness = np.apply_along_axis(func, 1, pop)
    best_idx = np.argmin(fitness)
    best_solution = pop[best_idx]
    best_score = fitness[best_idx]
    
    # Evolutionary loop
    for _ in range(max_iter):
        for i in range(pop_size):
            # Mutation: Select 3 random distinct individuals from the population (excluding i)
            indices = list(range(pop_size))
            indices.remove(i)
            a, b, c = pop[np.random.choice(indices, 3, replace=False)]
            
            # Create mutant vector
            mutant = a + F * (b - c)
            
            # Ensure mutant is within bounds
            mutant = np.clip(mutant, [bound[0] for bound in bounds], [bound[1] for bound in bounds])
            
            # Crossover
            crossover = np.random.rand(dimensions) < CR
            if not np.any(crossover):  # Ensure at least one dimension is selected for crossover
                crossover[np.random.randint(0, dimensions)] = True
            trial = np.where(crossover, mutant, pop[i])
            
            # Selection
            trial_score = func(trial)
            if trial_score < fitness[i]:
                pop[i] = trial
                fitness[i] = trial_score
                
                # Update best solution
                if trial_score < best_score:
                    best_solution = trial
                    best_score = trial_score
    
    return best_solution, best_score

# Example usage:
# Define an objective function
def objective(x):
    return np.sum(x**2)  # Sphere function (minima at 0)

# Define bounds for each dimension (e.g., [-5, 5] for each of 5 dimensions)
bounds = [(-5, 5)] * 5

# Run Differential Evolution
best_solution, best_score = differential_evolution(objective, bounds)
print("Best solution:", best_solution)
print("Best score:", best_score)


Best solution: [ 2.84997641e-28  7.09321296e-28 -5.43573989e-28 -1.38683116e-29
 -9.11068238e-28]
Best score: 1.7100707018820503e-54


In [8]:
test = [0,2,3,4,5]
r1, r2, r3 = np.random.choice(list(range(0,5)), 3, replace=False)

In [13]:
a = [2, 5.4, 1]
clipped = np.clip(a, 2, 5)
print(clipped)

[2. 5. 2.]
